# Week 3 : Oracle Based Quantum Algorithms

In this notebook we will provide qiskit codes for the algorithms discussed in the slides of chapter 3.

## Deutsch Algorithm

Deutsch algorithm is one of the first oracle based quantum algorithms. Given an 1-bit input 1-bit output Boolean function $f$, Deutsch algorithms gives a method to find if the function is balanced or constant. Here we implement the circuit for the Deutsch Algorithm. We assume that we are given an oracle for the function $f_s$ = [ $f$(0) , $f$(1) ] = [ 1 , 1 ] (which we will create ourselves).

In [1]:
from qiskit import *
from qiskit import Aer, BasicAer

In [2]:
#Building the oracle.
orcl = QuantumCircuit(2)
orcl.x(1)
oracle = orcl.to_instruction()

#Initializing quantum register, classical register and creating a quantum circuit.
q = QuantumRegister(2)
c = ClassicalRegister(1)
qc = QuantumCircuit(q,c)


#Building the circuit for Deutsch algorithm.
qc.x(q[1])
qc.h([q[0],q[1]])
qc.append(oracle, [q[0],q[1]])
qc.h([q[0],q[1]])
qc.measure(q[0],c[0])


#Executing the circuit and obtaining the results.
backend = Aer.get_backend("qasm_simulator")
qjob = execute(qc, backend)
counts = qjob.result().get_counts()
print(counts)

{'0': 1024}


We can see that the output is 0. Hence the function encoded in the oracle is constant which is indeed true since the function in the oracle is $f_s$ = [ 1 , 1 ].

### Deutsch-Jozsa Algorithm

In this section we will take time to implement the generaliztion of the Deutsch algorithm which is Deutsch-Jozsa algorithm (or shortly DJ algorithm). Given an n-bit Boolean function $f$ with the promise that $f$ is either balanced or constant, DJ algorithm outputs if $f$ is one or the other in just one oracle call.

For the implementation of the DJ algorithm, let us assume that we are given the oracle for the 3-bit Boolean funtion $f_s$ = [ 0, 1, 0, 1, 1, 0, 1, 0] (which again we will create ourselves).

In [3]:
#Building the oracle.
orcl2 = QuantumCircuit(5)
orcl2.x(0)
orcl2.ccx(0,2,3)
orcl2.x(0)
orcl2.x(2)
orcl2.ccx(0,2,3)
orcl2.x(2)
oracle2 = orcl2.to_instruction()


#Initializing quantum register, classical register and creating quantum circuit
q2 = QuantumRegister(5)
c2 = ClassicalRegister(3)
qc2 = QuantumCircuit(q2, c2)


#Building the circuit for DJ.
qc2.x(3)
qc2.h([0,1,2,3])
qc2.append(oracle2, q2)
qc2.h([0,1,2])
for i in range(3):
    qc2.measure(i,2-i)


#Executing the circuit and obtaining the results.
backend = BasicAer.get_backend("qasm_simulator")
qjob = execute(qc2, backend=backend, shots=100)
counts = qjob.result().get_counts()
print(counts)

{'101': 100}


Here we see that the output is non-zero and hence the function is balanced.

### Grover's Algorithm

Next we will implement Grover's algorithm on simulator. Despite availability of Grover class in qiskit aqua, to have a better understanding of the implementation of the algorithm, we will first implement Grover's algorithm from scratch and then we will compare the results with the results of the implementation using Grover class of qiskit.

In [4]:
#Constructing the oracle.
g_orcl = QuantumCircuit(3)
g_orcl.ccx(0,1,2)
g_oracle = g_orcl.to_instruction()

The above oracle marks the state $|11\rangle$ i.e, we flip the ancilla qubit from 0 to 1 if the state in the first register is $|11\rangle$. Let us assume that the oracle has been given. Let the marked item be our item of interest. Then the probability of obtaining the marked state without any iteration of Grover's algorithm is $\sin^{2}(\theta) = \sin^{2}(\pi/6) = 1/4$.

In [5]:
#Creating quantum register, classical register and quantm circuit
q3 = QuantumRegister(3)
c3 = ClassicalRegister(2)
c3_a = ClassicalRegister(1)
qc3 = QuantumCircuit(q3,c3_a, c3)


#Building the circuit that marks the item of interest.
qc3.h([0,1])
qc3.append(g_oracle,q3)
qc3.measure(q3[0],c3[1])
qc3.measure(q3[1],c3[0])
qc3.measure(q3[2],c3_a[0])


#Executing the circuit and obtaining the results.
backend = BasicAer.get_backend("qasm_simulator")
qjob = execute(qc3, backend=backend, shots=100)
counts = qjob.result().get_counts()
print(counts)

{'01 0': 19, '10 0': 28, '00 0': 30, '11 1': 23}


We can see that the ancilla bit (the single bit) is 1 with probability 1/4 as expected.

Now let us construct the circuit with one Grover's iteration.

In [6]:
#Creating quantum register, classical register and quantm circuit
q3 = QuantumRegister(3)
c3 = ClassicalRegister(2)
c3_a = ClassicalRegister(1)
qc3 = QuantumCircuit(q3,c3_a, c3)


#Constructing a circuit that marks all non-zero items.
mk_0 = QuantumCircuit(3)
mk_0.x([0,1])
mk_0.ccx(0,1,2)
mk_0.x([0,1])
mk_0.x(2)
mark_0 = mk_0.to_instruction()


#Building the circuit that marks the item of interest.
qc3.x(2)
qc3.h([0,1,2])
qc3.append(g_oracle, q3)
qc3.h([0,1])
qc3.append(mark_0, q3)
qc3.h([0,1])
qc3.h(2)


#Measuring the qubits.
qc3.measure(q3[0],c3[1])
qc3.measure(q3[1],c3[0])
qc3.measure(q3[2],c3_a[0])


from qiskit import Aer
#Executing the circuit and obtaining the results.
backend = Aer.get_backend("qasm_simulator")
qjob = execute(qc3, backend=backend)
counts = qjob.result().get_counts()
print(counts)

{'11 1': 1024}


We can see that after just one Grover iteration, the probability of obtaining the state $|11\rangle$ changes from $\sin(\pi/6) = 1/4$ to $\sin(3(\pi/6)) = \sin(\pi/2) = 1$.

### Simon's Algorithm

Given an oracle for a function $f$ with the promise that $f(x)=f(y)$ if and only if $x=y\oplus s$ for some fixed $s$, Simon's algorithm helps us find the shift $s$. Let us first construct an oracle of the function given by $f_s = [ 001, 010, 001, 010, 101, 111, 101, 111 ]$ where the shift $s$ is $010$.

In [7]:
#Constructing the oracle.
s_orcl = QuantumCircuit(6)
s_orcl.x([0,2])
s_orcl.ccx(0,2,5)
s_orcl.x(2)
s_orcl.ccx(0,2,4)
s_orcl.x(0)
s_orcl.x(2)
s_orcl.ccx(0,2,3)
s_orcl.ccx(0,2,5)
s_orcl.x(2)
s_orcl.ccx(0,2,3)
s_orcl.ccx(0,2,4)
s_orcl.ccx(0,2,5)

s_oracle = s_orcl.to_instruction()

Next we construct the circuit for Simon's algorithm.

In [8]:
#Creating quantum register, classical register and quantm circuit
q4 = QuantumRegister(6)
c4 = ClassicalRegister(3)
qc4 = QuantumCircuit(q4,c4)


#Implementing the Simon's algorithm
qc4.h([0,1,2])
qc4.append(s_oracle,q4)
qc4.h([0,1,2])


#Measuring the qubits
for i in range(3):
    qc4.measure(i,2-i)

    
#Executing the circuit and obtaining the results.
backend = Aer.get_backend("qasm_simulator")
qjob = execute(qc4, backend=backend,shots=10)
counts = qjob.result().get_counts()
print(counts)

{'100': 1, '001': 3, '101': 5, '000': 1}


We can see that we have obtained the states $|000\rangle, |001\rangle, |100\rangle$ and $|101\rangle$. So let us take $y_0=001$ and $y_1=100$. We cannot take the $y_i=101$ because 101 is a linear combination of $001$ and $100$ Solving for the system of equations $y.s$=0, we get two bits $s_0=0$ and $s_2=0$ of $s$. For the bit $s_1$, we can see that we have not obtained all possible values as output. Hence, the shift $s$ is non-zero. So we can conclude that the shift $s$ is $010$.